In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('100_Unique_QA_Dataset.csv')
print(df.head())

                                          question      answer
0                   What is the capital of France?       Paris
1                  What is the capital of Germany?      Berlin
2               Who wrote 'To Kill a Mockingbird'?  Harper-Lee
3  What is the largest planet in our solar system?     Jupiter
4   What is the boiling point of water in Celsius?         100


In [5]:
# Tokenize the sentence
def tokenize(text):
    text = text.lower()
    text = text.replace("?",'')
    text = text.replace("'","")
    return text.split()



In [6]:
# vocab
vocab = {'<UNK>':0}

In [7]:
def build_vocab(row):
    # print(row['question'],row['answer'])
    tokenized_question=tokenize(row['question'])
    tokenized_answer = tokenize(row['answer'])
    
    merged_token=tokenized_question + tokenized_answer
    for token in merged_token:
        if token not in vocab:
            vocab[token] = len(vocab)

In [8]:
df.apply(build_vocab,axis=1)

0     None
1     None
2     None
3     None
4     None
      ... 
85    None
86    None
87    None
88    None
89    None
Length: 90, dtype: object

In [9]:
# convert word into numerical indices
def text_to_indices(text,vocab):
    indexed_text = []
    for token in tokenize(text):
        if token in vocab:
            indexed_text.append(vocab[token])
        else:
            indexed_text.append(vocab['<UNK>'])
    return indexed_text


# text_to_indices("What is capital of France?",vocab)

In [10]:

import torch
from torch.utils.data import DataLoader,Dataset


In [11]:
class QADataset(Dataset):
    def __init__(self,df,vocab):
        self.df=df
        self.vocab = vocab
    
    def __len__(self,):
        return self.df.shape[0]

    def __getitem__(self, index):
        numerical_question = text_to_indices(self.df.iloc[index]['question'],self.vocab)
        numerical_answer = text_to_indices(self.df.iloc[index]['answer'],self.vocab)
        return torch.tensor(numerical_question),torch.tensor(numerical_answer)
    


In [12]:
dataset = QADataset(df,vocab)


In [13]:
dataloader = DataLoader(dataset,batch_size=1,shuffle=True)

In [14]:
for question,answer in dataloader:
    print(question,answer)

tensor([[42, 43, 44, 45, 46, 47, 48]]) tensor([[49]])
tensor([[42, 18,  2, 62, 63,  3, 64, 18]]) tensor([[65]])
tensor([[  1,  87, 229, 230, 231, 232]]) tensor([[233]])
tensor([[  1,   2,   3,   4,   5, 113]]) tensor([[114]])
tensor([[  1,   2,   3, 234,   5, 235]]) tensor([[131]])
tensor([[ 42, 167,   2,   3,  17, 168, 169]]) tensor([[170]])
tensor([[ 42,   2,   3, 274, 211, 275]]) tensor([[276]])
tensor([[10, 29,  3, 30, 31]]) tensor([[32]])
tensor([[ 10,  75, 208]]) tensor([[209]])
tensor([[  1,   2,   3, 146,  86,  19, 192, 193]]) tensor([[194]])
tensor([[ 42, 255,   2, 256,  83, 257, 258]]) tensor([[259]])
tensor([[ 1,  2,  3, 92, 93, 94]]) tensor([[95]])
tensor([[ 10, 140,   3, 141, 270,  93, 271,   5,   3, 272]]) tensor([[273]])
tensor([[ 42,   2,   3, 210, 137, 168, 211, 169]]) tensor([[113]])
tensor([[ 42, 216, 118, 217, 218,  19,  14, 219,  43]]) tensor([[220]])
tensor([[ 10,  11, 189, 158, 190]]) tensor([[191]])
tensor([[ 78,  79, 195,  81,  19,   3, 196, 197, 198]]) tensor(

In [15]:
# Architecture of RNN
import torch.nn as nn


In [16]:
class SimpleRNN(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size,embedding_dim=50)
        self.rnn =nn.RNN(50,64,batch_first=True)
        self.fc= nn.Linear(64,vocab_size)
        
        

    def forward(self,question):
        embedded_question = self.embedding(question)
        hidden,final= self.rnn(embedded_question)
        output = self.fc(final.squeeze(0))
        return output
        


In [17]:
# x= nn.Embedding(324,embedding_dim=0)
# y = nn.RNN(50,60,batch_first=True)

In [18]:
learning_rate = 0.001
epochs =100

In [19]:
model = SimpleRNN(len(vocab))

In [20]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr =learning_rate)

In [21]:
# training Loop
for epoch in range(epochs):
    total_loss = 0

    for question,answer in dataloader:
        optimizer.zero_grad()

        # forward pass
        output = model(question)

        # loss calculate
        loss = criterion(output,answer[0])

        # gradients
        loss.backward()

        # update
        optimizer.step()

        total_loss = total_loss+loss.item()

    print(f"Epoch {epoch+1} , Loss: {total_loss:4f}")



Epoch 1 , Loss: 522.601129
Epoch 2 , Loss: 457.242719
Epoch 3 , Loss: 379.274297
Epoch 4 , Loss: 316.719943
Epoch 5 , Loss: 265.864625
Epoch 6 , Loss: 218.690829
Epoch 7 , Loss: 174.248908
Epoch 8 , Loss: 136.680828
Epoch 9 , Loss: 105.167872
Epoch 10 , Loss: 80.261959
Epoch 11 , Loss: 62.111665
Epoch 12 , Loss: 48.386006
Epoch 13 , Loss: 38.238930
Epoch 14 , Loss: 31.275081
Epoch 15 , Loss: 25.782332
Epoch 16 , Loss: 21.627101
Epoch 17 , Loss: 18.458121
Epoch 18 , Loss: 15.771088
Epoch 19 , Loss: 13.545308
Epoch 20 , Loss: 11.798768
Epoch 21 , Loss: 10.496070
Epoch 22 , Loss: 9.012820
Epoch 23 , Loss: 7.922178
Epoch 24 , Loss: 7.044049
Epoch 25 , Loss: 6.290388
Epoch 26 , Loss: 5.653000
Epoch 27 , Loss: 5.103973
Epoch 28 , Loss: 4.626724
Epoch 29 , Loss: 4.207681
Epoch 30 , Loss: 3.838425
Epoch 31 , Loss: 3.522766
Epoch 32 , Loss: 3.248671
Epoch 33 , Loss: 2.988909
Epoch 34 , Loss: 2.762930
Epoch 35 , Loss: 2.558385
Epoch 36 , Loss: 2.372990
Epoch 37 , Loss: 2.206495
Epoch 38 , Loss: 

In [26]:
# #  Predicition
# def predict(model,question,threshold=0.8):

#     # convert question to number
#     numerical_question = text_to_indices(question,vocab)

#     # convert to tensor
#     question_tensor = torch.tensor(numerical_question).unsqueeze(0)

#     # send to model
#     output = model(question_tensor)

#     # converts logits to probability
#     probs = torch.nn.functional.softmax(output,dim=1)

#     # find max probability
#     value,index = torch.max(probs,dim=1)

#     if value < threshold:
#         print("I don't know")
    
#     return list(vocab.keys())[index]

def predict(model, question, threshold=0.8):
    """
    Predict the answer for a given question using the model.
    
    Parameters:
        model: trained PyTorch model
        question: str, the input question
        threshold: float, minimum probability required to return an answer
    
    Returns:
        predicted answer as a string or "I don't know" if confidence is below threshold
    """
    # convert question to numerical indices
    numerical_question = text_to_indices(question, vocab)

    # convert to tensor
    question_tensor = torch.tensor(numerical_question).unsqueeze(0)

    # send to model
    output = model(question_tensor)

    # convert logits to probabilities
    probs = torch.nn.functional.softmax(output, dim=1)

    # find max probability and index
    value, index = torch.max(probs, dim=1)

    # check threshold
    if value.item() < threshold:
        return "I don't know"
    
    # return predicted word/label
    return list(vocab.keys())[index.item()]



In [29]:
question = input("Enter your question")
answer = predict(model,question).capitalize()
print(answer)

I don't know
